### Imports

In [1]:
import customized_headlines.data.load as load
import re
import pandas as pd
import preprocessor as p
from gensim.utils import deaccent
from emoji import demojize

### Load data

In [2]:
data_raw = load.data_raw
data_processed = load. data_processed

### Read data

In [3]:
# Read csv file as dataframe
df = pd.read_csv(f'{data_raw}/extracted_news.csv')

# Print dataframe shape
print(df.shape)

# Show dataframe
df.head(3)

(100, 20)


,url,index,resolved,status,datetime_utc,error,filename,mimetype,encoding,extract_error,canonical_url,title,description,raw_content,comments,author,categories,tags,date,sitename
0,https://unamglobal.unam.mx/comunidades-indigen...,46,NaN,200,2023-04-02T15:25:07.419616,NaN,16dd649be430106e823d0f6ad1d7c638.html,text/html,utf-8,NaN,https://unamglobal.unam.mx/comunidades-indigen...,Comunidades indígenas urbanas en tiempos de pa...,La pandemia derivada de la COVID-19 ha signifi...,La pandemia derivada de la COVID-19 ha signifi...,NaN,Beto Torres,BLOG|Opinión,blog|opinión|blog|opinión,2021-02-09,UNAM Global
1,https://www.milenio.com/negocios/larry-rubin-t...,0,NaN,200,2023-04-02T15:25:07.112501,NaN,bf6c8a5bb799314dcdb37c7085c38a81.html,text/html,utf-8,NaN,https://www.milenio.com/negocios/larry-rubin-t...,Larry Rubin: Próximo titular de Economía deber...,"Larry Rubin, presidente de la American Society...",El presidente de la American Society of Mexico...,NaN,Eduardo de la Rosa,Negocios,American Society of Mexico|tatiana clouthier|a...,2022-10-06,Grupo Milenio
2,https://unamglobal.unam.mx/el-giro-y-el-miedo-...,47,NaN,200,2023-04-02T15:25:09.140811,NaN,99ef9ca8a1832cd7de1d7cf6cb6d54c8.html,text/html,utf-8,NaN,https://unamglobal.unam.mx/el-giro-y-el-miedo-...,El Giro y el miedo a la bici | UNAM Global,Hace unos días se celebró en Turín la centésim...,Hace unos días se celebró en Turín la centésim...,NaN,Beto Torres,BLOG|Opinión,blog|opinión|blog|opinión,2021-05-19,UNAM Global


### Process

In [4]:
# Remove urls
p.set_options(p.OPT.URL)
df['clean_content'] = df['raw_content'].apply(lambda x: p.clean(x))

# Remove accents
df['clean_content'] = df['clean_content'].apply(lambda x: deaccent(x))

# Tokenize mentions and hashtags
p.set_options(p.OPT.MENTION, p.OPT.HASHTAG)
df['clean_content'] = df['clean_content'].apply(lambda x: p.tokenize(x))

# Remove two or more question marks together
df['clean_content'] = df['clean_content'].apply(lambda x : re.sub(r'\?{2,}', '', x))

# Replace emojis with descriptions
df['clean_content'] = df['clean_content'].apply(lambda x: demojize(x))

In [5]:
# Create new column with title and content
df["combined"] = f"Titulo: {df.title.str.strip()} Contenido: {df.clean_content.str.strip()}"

### Export

In [6]:
df.to_csv(f'{data_processed}/news.csv', index=False)